In [5]:
!pip install pandas pymongo

https://somjang.tistory.com/entry/MongoDB-Python%EA%B3%BC-Pymongo%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%B6%94%EA%B0%80%ED%95%98%EA%B3%A0-%EC%B6%9C%EB%A0%A5%ED%95%B4%EB%B3%B4%EA%B8%B0

In [3]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import time

In [4]:
client = MongoClient()
db = client['lol']

In [5]:
match_col = db['match']
matches = match_col.find()

In [ ]:
tier_col = db['tier']
#tiers = tier_col.find()

# Features

In [6]:
part_info = ['summonerName', 'puuid', 'summonerLevel', 'win']

f_objectives = ['damageDealtToObjectives', 'baronKills', 'dragonKills','objectivesStolen', 'objectivesStolenAssists',
                'neutralMinionsKilled']

f_towers = ['firstTowerKill', 'firstTowerAssist','inhibitorKills','turretKills']

f_damages = ['damageDealtToTurrets', 'damageSelfMitigated','magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt','physicalDamageDealtToChampions', 'physicalDamageTaken','totalDamageDealt', 'totalDamageDealtToChampions',
             'totalDamageShieldedOnTeammates', 'totalDamageTaken','trueDamageDealt', 'trueDamageDealtToChampions', 
             'trueDamageTaken', 'totalHealsOnTeammates','totalHeal',   'totalTimeCCDealt','totalUnitsHealed']

f_sight = ['detectorWardsPlaced', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced','visionScore']

f_champ = ['championName', 'champLevel', 'champExperience', 'individualPosition','role', 'lane','totalMinionsKilled']

f_kdas = ['kills', 'deaths', 'assists',  'firstBloodKill','firstBloodAssist','doubleKills','tripleKills','quadraKills', 'pentaKills',
         'unrealKills', 'bountyLevel','killingSprees']
f_gold = ['goldEarned', 'goldSpent', 'consumablesPurchased', 'itemsPurchased']

feature_list = []
for li in [part_info, f_kdas, f_objectives, f_towers, f_damages, f_sight, f_champ, f_gold]:
    feature_list.extend(li)
    
print(feature_list,'\n', len(feature_list))

['summonerName', 'puuid', 'summonerLevel', 'win', 'kills', 'deaths', 'assists', 'firstBloodKill', 'firstBloodAssist', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'unrealKills', 'bountyLevel', 'killingSprees', 'damageDealtToObjectives', 'baronKills', 'dragonKills', 'objectivesStolen', 'objectivesStolenAssists', 'neutralMinionsKilled', 'firstTowerKill', 'firstTowerAssist', 'inhibitorKills', 'turretKills', 'damageDealtToTurrets', 'damageSelfMitigated', 'magicDamageDealt', 'magicDamageDealtToChampions', 'magicDamageTaken', 'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt', 'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken', 'totalHealsOnTeammates', 'totalHeal', 'totalTimeCCDealt', 'totalUnitsHealed', 'detectorWardsPlaced', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced', 'visionScore', 'championName', 'champLevel', 'cha

# Load tiers

In [ ]:
def load_tier(id):
    entity = tier_col.find_one(filter={'_id': id})
    tier, lp, winrate = entity['tier'], entity['leaguePoints'], entity['winrate']
    return tier, lp, winrate
    

# Preparing

In [8]:
#N = matches.count()
batch = 10000
fname = time.strftime("%m%d-%H%M.csv")

i = 0
Dic = []
for match in matches:
    if match['info']['gameMode'] == 'CLASSIC':
        if i%batch == 0 and i != 0:
            df = pd.DataFrame(Dic)
            df.to_csv('data/'+fname, index=False, encoding='UTF-8')
            Dic = []

        for part in match['info']['participants']:
            if part['lane'] == 'TOP' and part['teamPosition'] == 'TOP':
                dic = {}
                dic['gameId'] = match['info']['gameId']
                dic['gameDuration'] = match['info']['gameDuration']
                tier, lp, winrate = load_tier(part['summonerId'])
                dic['tier'] = tier
                dic['leaguePoints'] = lp
                dic['winrate'] = winrate
                for f in feature_list:
                    dic[f] = part[f]
                Dic.append(dic)
        i += 1

df = pd.DataFrame(Dic)
df.to_csv('data/'+fname, index=False, encoding='UTF-8')
Dic = []

